In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eeg-eye-state-dataset/train.csv
/kaggle/input/eeg-eye-state-dataset/test.csv


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("syednazmussakib/eeg-eye-state-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/eeg-eye-state-dataset


In [3]:
train=pd.read_csv("/kaggle/input/eeg-eye-state-dataset/train.csv")

In [4]:
train.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,target,id
0,4313.33,4008.72,4261.03,4090.26,4342.56,4626.15,4078.46,4633.85,4237.95,4274.87,4236.41,4304.62,4633.33,4386.67,0,1
1,4316.92,4016.92,4262.56,4101.03,4344.62,4632.31,4082.05,4643.08,4248.21,4286.15,4250.26,4311.79,4642.56,4390.77,0,2
2,4321.03,4018.46,4262.56,4107.69,4345.13,4639.49,4088.72,4645.64,4251.79,4284.10,4248.21,4312.82,4640.51,4387.69,0,3
3,4318.97,4009.23,4265.13,4102.56,4339.49,4637.95,4090.26,4649.23,4255.90,4283.08,4240.00,4311.79,4635.90,4369.23,0,4
4,4305.13,3997.95,4265.64,4093.85,4331.79,4635.90,4088.72,4650.26,4252.31,4287.69,4237.44,4308.21,4628.72,4355.38,0,5


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AF3     9980 non-null   float64
 1   F7      9980 non-null   float64
 2   F3      9980 non-null   float64
 3   FC5     9980 non-null   float64
 4   T7      9980 non-null   float64
 5   P7      9980 non-null   float64
 6   O1      9980 non-null   float64
 7   O2      9980 non-null   float64
 8   P8      9980 non-null   float64
 9   T8      9980 non-null   float64
 10  FC6     9980 non-null   float64
 11  F4      9980 non-null   float64
 12  F8      9980 non-null   float64
 13  AF4     9980 non-null   float64
 14  target  9980 non-null   int64  
 15  id      9980 non-null   int64  
dtypes: float64(14), int64(2)
memory usage: 1.2 MB


from the above iformation we can see that our we do not need father preprocessing of data ,since it is in the perfect data type format

In [6]:
train.shape

(9980, 16)

In [7]:
train.isna().sum()

AF3       0
F7        0
F3        0
FC5       0
T7        0
P7        0
O1        0
O2        0
P8        0
T8        0
FC6       0
F4        0
F8        0
AF4       0
target    0
id        0
dtype: int64

having no null values we can go a and head check our target values if they are for binary classification or multiclass clasification

In [8]:
train["target"].unique()

array([0, 1])

In [9]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [10]:
cols = train.columns
cols

Index(['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6',
       'F4', 'F8', 'AF4', 'target', 'id'],
      dtype='object')

In [11]:
test=pd.read_csv("/kaggle/input/eeg-eye-state-dataset/test.csv")

In [12]:
#we are going to use MinMaxScaler to scale our data before feeding it into the models
mc=MinMaxScaler()

X=train.drop("target",axis=1)
y=train["target"]

xtrain,xval,ytrain,yval=train_test_split(X,y,test_size=.2,stratify=y)

In [13]:
xtrain_norm=mc.fit_transform(xtrain)
xval_norm=mc.transform(xval)

In [14]:
lg=LogisticRegression(
    penalty='l2',             # Regularization: 'l1', 'l2', 'elasticnet', or 'none'
    dual=False,               # Dual or primal formulation (only for l2 penalty)
    tol=1e-4,                 # Tolerance for stopping criteria
    C=1.0,                    # Inverse of regularization strength (smaller = stronger regularization)
    fit_intercept=True,       # Whether to include the intercept in the model
    intercept_scaling=1,      # Scaling factor for intercept in 'liblinear' solver
    solver='lbfgs',           # Optimization solver: 'lbfgs', 'liblinear', 'sag', 'saga', 'newton-cg'
    max_iter=100,             # Maximum number of iterations for convergence
    multi_class='auto',       # Multi-class strategy: 'auto', 'ovr', 'multinomial'
    verbose=0,                # Verbosity level
    n_jobs=None,              # Number of jobs for parallel computation
    random_state=None         # Random seed
)


In [15]:
svc=SVC(
    C=1.0,                    # Regularization parameter (smaller = stronger regularization)
    kernel='rbf',             # Kernel type: 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
    degree=3,                 # Degree for 'poly' kernel
    gamma='scale',            # Kernel coefficient: 'scale', 'auto', or a float
    coef0=0.0,                # Independent term for 'poly' and 'sigmoid' kernels
    shrinking=True,           # Whether to use the shrinking heuristic
    probability=False,        # Enable probability estimates
    tol=1e-3,                 # Tolerance for stopping criteria
    cache_size=200,           # Cache size (in MB) for kernel
    class_weight=None,        # Weights for classes: 'balanced' or dict
    verbose=False,            # Verbosity level
    max_iter=-1,              # Maximum number of iterations (-1 for no limit)
    random_state=None         # Random seed
)


In [16]:
xgb=XGBClassifier(
    n_estimators=100,          # Number of boosting rounds
    max_depth=6,               # Maximum depth of a tree
    learning_rate=0.3,         # Boosting learning rate
    subsample=1.0,             # Subsample ratio of training samples
    colsample_bytree=1.0,      # Subsample ratio of columns for each tree
    colsample_bylevel=1.0,     # Subsample ratio of columns for each split
    colsample_bynode=1.0,      # Subsample ratio of columns for each node
    min_child_weight=1,        # Minimum sum of instance weight (hessian) in a child
    gamma=0,                   # Minimum loss reduction to make a split
    reg_alpha=0,               # L1 regularization term
    reg_lambda=1,              # L2 regularization term
    scale_pos_weight=1,        # Balancing of positive and negative weights
    objective='binary:logistic', # Learning objective
    booster='gbtree',          # Booster type: 'gbtree', 'gblinear', 'dart'
    verbosity=1,               # Verbosity level (0 = silent)
    random_state=None          # Random seed
)


In [17]:
rf=RandomForestClassifier(
    n_estimators=100,          # Number of trees in the forest
    criterion='gini',          # Splitting criterion: 'gini' or 'entropy'
    max_depth=None,            # Maximum depth of the trees
    min_samples_split=2,       # Minimum samples required to split an internal node
    min_samples_leaf=1,        # Minimum samples required to be at a leaf node
    min_weight_fraction_leaf=0.0, # Minimum weighted fraction of the sum total of weights at a leaf
    max_features='sqrt',       # Number of features to consider when looking for the best split
    max_leaf_nodes=None,       # Maximum number of leaf nodes
    bootstrap=True,            # Whether to use bootstrapped samples
    oob_score=False,           # Whether to use out-of-bag samples for scoring
    n_jobs=None,               # Number of jobs for parallel computation
    random_state=None,         # Random seed
    verbose=0,                 # Verbosity level
    warm_start=False,          # Reuse previous solution to speed up training
    class_weight=None          # Weights for classes: 'balanced', 'balanced_subsample', or dict
)


In [18]:
scores = cross_val_score(lg,
                         X=xtrain_norm,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.763 +/- 0.013


In [19]:
scores = cross_val_score(xgb,
                         X=xtrain_norm,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.994 +/- 0.003


In [20]:
scores = cross_val_score(rf,
                         X=xtrain_norm,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.991 +/- 0.004


In [21]:
scores = cross_val_score(svc,
                         X=xtrain_norm,
                         y=ytrain,
                         cv=10,
                         n_jobs=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.757 +/- 0.015


form the above we can see thet xgb and rf models did agread job in learning the data

In [22]:
error=[]
xgb.fit(xtrain_norm,ytrain)
xval_pred=xgb.predict(xval_norm)
for i in range(len(xval)):
    if xval_pred[i] != yval.iloc[i]:
        error.append(i)

print(f"we have {len(error)} misclassification out of {len(xval_pred)}")
print(f'accuracy:{1-len(error)/len(xval_pred)}')

we have 13 misclassification out of 1996
accuracy:0.9934869739478958


In [23]:
error=[]
rf.fit(xtrain_norm,ytrain)
xval_pred=xgb.predict(xval_norm)
for i in range(len(xval)):
    if xval_pred[i] != yval.iloc[i]:
        error.append(i)

print(f"we have {len(error)} misclassification out of {len(xval_pred)}")
print(f'accuracy:{1-len(error)/len(xval_pred)}')

we have 13 misclassification out of 1996
accuracy:0.9934869739478958


In [24]:
xtest=test.drop("target",axis=1)
ytest=test['target']
xtest_norm=mc.transform(xtest)

In [25]:
error_test=[]
xtest_pred=xgb.predict(xtest_norm)
for i in range(len(xtest_pred)):
    if xtest_pred[i] != ytest.iloc[i]:
        error_test.append(i)

print(f"we have {len(error_test)} misclassification out of {len(xtest_pred)}")
print(f'accuracy:{1-len(error_test)/len(xtest_pred)}')

we have 2459 misclassification out of 5000
accuracy:0.5082


In [26]:
error_test=[]
xtest_pred=rf.predict(xtest_norm)
for i in range(len(xtest_pred)):
    if xtest_pred[i] != ytest.iloc[i]:
        error_test.append(i)

print(f"we have {len(error_test)} misclassification out of {len(xtest_pred)}")
print(f'accuracy:{1-len(error_test)/len(xtest_pred)}')

we have 2184 misclassification out of 5000
accuracy:0.5631999999999999
